In [ ]:
function [th_hat, P_min, P, numberit, parameter] = ekf_toa_robust(r_ges,theta_init,BS,parameter)
% The function computes the robust EKF with ToA estimates based on M-estimation. 
%
%% INPUTS
% r_ges:    measured distances (M x N)
% theta_init:  initial state estimate
% BS:   base station positions
%
%% OUTPUTS
% th_hat:             state estimates
% P_min:              apriori covariance
% P:                  aposteriori covariance
%%  created by Michael Muma
%   Based on a function written by Ulrich Hammes, Signal Processing Group, TU Darmstadt, February 2009
%   version 30 May 2018
%   When using code, cite our work:
%
%   "Robust Statistics for Signal Processing"
%   Zoubir, A.M. and Koivunen, V. and Ollila, E. and Muma, M.
%   Cambridge University Press, 2018.
%
%   and 
%
%  "Robust Tracking and Geolocation for Wireless Networks in NLOS Environments." 
%   Hammes, U., Wolsztynski, E., and Zoubir, A.M.
%   IEEE Journal on Selected Topics in Signal Processing, 3(5), 889-901, 2009.


In [8]:
import scipy.io
import scipy as sp
import numpy as np

r_ges = scipy.io.loadmat('r_ges.mat',struct_as_record=False)['r_ges']
BS = scipy.io.loadmat('BS.mat',struct_as_record=False)['BS']
theta_init = scipy.io.loadmat('theta_init.mat',struct_as_record=False)['theta_init']

In [30]:
def ekf_toa_robust(r_ges, theta_init, BS, parameter={}):
    
    M  = len(BS) # M numer of BS, N number of samples
    N  = len(r_ges[1,:])
    
    x  = BS[:,1]
    y  = BS[:,1]
    
    if len(parameter) == 0:
        # use default parameters
        print("parameters are set to default")
        sigma_v = 1
        P0 = np.diag([100, 100, 10, 10]) # initial state covariance
        R  = 150**2 * np.diag(np.ones(M)) # measurement covariance
        Ts = 0.2 # sampling frequency
        A  = np.array([[1, 0, Ts, 0], \
                       [0, 1, 0, Ts], \
                       [0, 0, 1,  0], \
                       [0, 0, 0,  1]])
        Q  = sigma_v **2 *np.eye(2)
        G  = np.vstack([Ts**2/2*np.eye(2), Ts*np.eye(2) ])
    else:
        P0 = parameter['P0']
        R  = parameter['R']
        Q  = parameter['Q']
        G  = parameter['G']
        A  = parameter['A']
    if len(BS) != len(theta_init[:,0]) or len(BS) != len(P0[:,0]):
        raise Exception('State vector or state covariance do not match the dimensions of the BS')
        
    P = np.zeros((N,4,4))
    P[0,:,:] = P0
    th_hat = np.zeros((4,N))
    th_hat[:,0] = theta_init.flatten()
    th_hat_min = np.zeros([4,N])
    P_min = np.zeros([N,4,4])
    H = np.zeros((M,4))
    h_min = np.zeros(M)
    sigma2 = np.zeros(N)
    # numberit = np.zeros(N)
    
    for kk in range(1,N+1):
        th_hat_min[:,kk] = A @ th_hat[:,kk-1]
        
        for ii in range(M):
            H[ii,:] = [(th_hat_min[0,kk]-x[ii])/\
           np.sqrt((th_hat_min[0,kk]-x[ii])**2 \
                   + (th_hat_min[1,kk]-y[ii])**2) ,\
           (th_hat_min[1,kk]-y[ii])/\
           np.sqrt((th_hat_min[0,kk]-x[ii])**2 \
                   + (th_hat_min[1,kk]-y[ii])**2)\
           ,0,0]
            h_min[ii] = np.sqrt((th_hat_min[0,kk]-x[ii])**2 \
                               + (th_hat_min[1,kk]-y[ii])**2)
            
        P_min[kk,:,:] = A@P[kk-1,:,:]@A.T + G@Q@G.T
        
        # measurement residuals
        vk = r_ges[:,kk] - h_min.T
        
        Psi = sp.linalg.block_diag(P_min[kk,:,:],R)
        try:
            C = sp.linalg.cholesky(Psi)
        except:
            Psi = Psi + np.eye(M+4)*0.1
            
        S  = np.linalg.inv(C.T) @ np.vstack([np.eye(4), H])
        rk = np.linalg.inv(C.T) @ np.vstack([th_hat_min[:,kk],\
                                            r_ges[:,kk]-h_min.T + H @ th_hat_min[:,kk]])
        th_hat[:,kk] = np.linalg.pinv(S) @ rk
        
        #Todo
        th_hat[:,kk] = m_param_est(rk,S,th_hat[:,kk],parameter)
        
        # robust covariance estimate
        if parameter['var_est'] == 1:
            # update for robust covariance estimation
            for ii in range(M):
                h_min[ii] = np.sqrt( (th_hat[0,kk] - x[ii])**2 + \
                                   (th_hat[1,kk] - y[ii])**2)
            dd = r_ges[:,kk] - h_min.T
            sigma = 1.483*np.median(abs(dd-np.median(dd)))
            sigma2[kk] = sigma**2
            R = sigma2[kk] @ np.eye(M)
            
        K = P_min[kk,:,:] @ H.T @ np.linalg.inv(H@P_min[kk,:,:]@H.T+R)
        P[kk,:,:] = (np.eye(4) - K@H) @ P_min[kk,:,:]
        
    parameter['Rest'] = sigma2

In [31]:
ekf_toa_robust(r_ges, theta_init, BS, parameter={})

parameters are set to default
10
4
4


Exception: State vector or state covariance do not match the dimensions of the BS

In [13]:
a = np.array([[1,3,4],[2,5,6],[3,3,3]])
a.I

AttributeError: 'numpy.ndarray' object has no attribute 'I'

In [20]:
i = {}
len(i)

0

In [35]:
print(BS.shape[0])
print(len(theta_init[:,0]))
theta_init

10
4


array([[1],
       [0],
       [0],
       [0]], dtype=uint8)